# 一、批量 ASR 转写

In [25]:
import pandas as pd, whisper, tqdm, os
from opencc import OpenCC

In [26]:
df = pd.read_csv("/home/zhaoshuai/workspace_multimodal/data_map.csv")
model = whisper.load_model("base", device="cpu")  
cc = OpenCC('t2s')  # 繁转简

In [27]:
def transcribe_to_simplified(path):
    r = model.transcribe(path, language="zh", task="transcribe")
    text = r["text"].strip()
    text_simplified = cc.convert(text)  # 转成简体
    return text_simplified

In [ ]:
asr_results = []
for p in tqdm.tqdm(df["audio_path"]):
    r = transcribe_to_simplified(p)
    asr_results.append(r.strip())

In [ ]:
df["asr_text"] = asr_results
df.to_csv("map_with_asr.csv", index=False)

['GYXX光影行星週四次一雙艦包南背包技能旅行大容量汽車齊行電腦收包']